In [36]:
# imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as mpl

import itertools

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from chap7 import PurgedKFold
from chap7 import cvScore

#ライブラリnumbaのインストールが必要
#pip install numba

# 8.6　人工データによる実験

## スニペット8.7 人工データセットの作成


In [37]:
def getTestData(n_features=40, n_informative=10, n_redundant=10, n_samples=10000):
    #ランダムなデータセットを作成
    from sklearn.datasets import make_classification
    trnsX, cont = make_classification(
        n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_redundant=n_redundant, random_state=0, shuffle=False
    )
    #df0 = pd.DatetimeIndex(periods=n_samples, freq=pd.tseries.offsets.BDay(), end = pd.datetime.today())
    df0 = pd.bdate_range(end=pd.Timestamp.today(), periods=n_samples)

    trnsX, cont= pd.DataFrame(trnsX,index=df0), pd.Series(cont,index=df0).to_frame('bin')

    #xrangeは古いのでrangeを使用
    df0 = ['I_' + str(i) for i in range(n_informative)] + ['R_' + str(i) for i in range(n_redundant)]
    df0 += ['N_' + str(i) for i in range(n_features - len(df0))]

    trnsX.columns = df0

    cont['w'] = 1./cont.shape[0]
    cont['t1'] = pd.Series(cont.index, index=cont.index)

    return trnsX, cont

    バージョンアップのため変更あり

xrange→range


df0 = pd.DatetimeIndex(periods=n_samples, freq=pd.tseries.offsets.BDay(), end = pd.datetime.today())

    
df0 = pd.bdate_range(end=pd.Timestamp.today(), periods=n_samples)

解説

bdate_rangeはその期間の営業日を取得

https://pandas.pydata.org/docs/dev/reference/api/pandas.bdate_range.html

make_classification：分類問題を作成

binが答え

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html

https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators


In [38]:
trnsX, cont = getTestData()

In [39]:
trnsX

,I_0,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8,I_9,...,N_10,N_11,N_12,N_13,N_14,N_15,N_16,N_17,N_18,N_19
1985-03-25,2.843740,0.456554,0.171107,-4.511382,0.278990,-3.474726,2.955550,2.698865,1.542440,2.198168,...,-0.330515,-0.845502,-1.477466,1.217536,0.304644,1.557365,0.202843,0.160110,0.933805,-0.132272
1985-03-26,3.561541,-1.566097,3.342813,-1.938909,2.075749,-3.486711,0.494908,0.309615,1.059439,-0.792433,...,-0.020384,-0.751467,0.212077,0.285038,0.125461,0.203534,-0.376495,-0.938780,-0.142879,0.533263
1985-03-27,7.699248,-3.030124,-0.859302,-0.033351,1.113719,-0.877844,2.344033,4.089113,2.287786,0.611413,...,0.744056,0.914181,1.586483,0.692802,-0.953431,0.679360,0.565153,0.219302,-1.110504,-1.086061
1985-03-28,-0.149801,-3.182187,2.695894,1.359997,2.992416,-0.417971,-1.214058,1.268313,-3.720913,-2.580578,...,-1.960632,-2.064914,1.258648,-1.031856,0.645146,-0.063900,0.305844,0.371489,3.218969,0.867178
1985-03-29,-2.157903,0.046380,0.697217,-1.012036,1.856002,-2.311465,2.715493,0.444433,-1.921790,-2.472372,...,-0.841121,0.081347,-2.587682,-0.416436,-1.077859,-0.428086,-0.183735,-0.434254,-2.124955,-0.709056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-17,2.489902,-0.523701,0.618210,-1.837222,1.905865,0.051481,-2.072887,1.133835,0.055833,-1.655887,...,-0.013532,-0.215027,0.869924,-0.932547,0.644979,0.175920,1.353286,-1.436413,-0.741075,1.054140
2023-07-18,-0.742495,1.595855,2.173729,1.420993,-1.619853,1.952217,-2.131979,-2.340327,-1.629928,0.080561,...,0.962720,-0.191763,2.788231,0.055138,0.807920,0.457464,1.129258,0.273195,1.419955,-1.095283
2023-07-19,-0.420406,-0.650531,3.524902,-0.608855,-0.823038,0.070312,-1.619873,-1.560697,-0.939518,-1.471037,...,-0.176512,0.676007,0.878948,1.322274,-0.794662,0.646342,0.464202,-1.220561,0.101444,0.455718
2023-07-20,-0.697229,-1.974764,0.438944,-0.208492,-0.279182,-1.887733,-1.931841,-1.163289,-0.111943,-0.732073,...,1.179474,-0.825594,-0.419181,1.142775,-0.696596,-0.018242,0.090638,1.535604,-0.857168,0.433795


In [40]:
cont

,bin,w,t1
1985-03-25,0,0.0001,1985-03-25
1985-03-26,0,0.0001,1985-03-26
1985-03-27,0,0.0001,1985-03-27
1985-03-28,0,0.0001,1985-03-28
1985-03-29,0,0.0001,1985-03-29
...,...,...,...
2023-07-17,1,0.0001,2023-07-17
2023-07-18,1,0.0001,2023-07-18
2023-07-19,1,0.0001,2023-07-19
2023-07-20,1,0.0001,2023-07-20


binが分類の答え

wはよくわからないが、たぶんウェイト

t1は日付（インデックスと同じ）

dataに保存しておく

In [41]:
#trnsX.to_csv("./data/chap8_trnsX.csv")
#cont.to_csv("./data/chap8_cont.csv")

# 8.3 代替効果による特徴量重要度



## スニペット8.2 MDI特徴量重要性

In [42]:
def featImpMDI(fit, featNames):
    #ISで平均不純度減少量に基づく特徴量重要度
    df0 = {i:tree.feature_importances_ for i,tree in enumerate (fit.estimators_)}
    df0 = pd.DataFrame.from_dict(df0, orient='index')
    df0.columns = featNames
    df0 = df0.replace(0,np.nan) # max_features=1のため
    imp = pd.concat({'mean': df0.mean(), 'std': df0.std()*df0.shape[0]**-.5}, axis=1)
    imp /= imp['mean'].sum()

    return imp

人工データでやってみる

スニペット8.8を編集し、サンプルデータにフィットする関数を作成

分類器として決定木を使用


In [43]:
def fit_sample_data(trnsX, cont, n_estimators=1000, cv=10, max_samples=1., numThreads=24, pctEmbargo=0, scoring='accuracy', method='SFI', minWLeaf=0., **kargs):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier

    #謎のライブラリをインポートしているのでコメントアウトしておく
    #from mpEngine import mpPandas0bj

    n_jobs = (-1 if numThreads>1 else 1) #dirac1のht_helperの場合は1スレッドのみをはしらせる

    # 1) classifierとcvを用意する。
    # マスキングを避けるためにmax_features=1とする

    clf = DecisionTreeClassifier(criterion='entropy', max_features=1, class_weight='balanced', min_weight_fraction_leaf=minWLeaf)

    #base_estimatorはestimatorに変わった
    #clf = BaggingClassifier(base_estimator=clf, n_estimators=n_estimators, max_features=1., max_samples=max_samples, oob_score=True, n_jobs=n_jobs)
    clf = BaggingClassifier(base_estimator=clf, n_estimators=n_estimators, max_features=1., max_samples=max_samples, oob_score=True, n_jobs=n_jobs)

    fit = clf.fit(X = trnsX, y=cont['bin'], sample_weight=cont['w'].values)
    oob = fit.oob_score_

    return fit, oob, clf


In [44]:
fit, oob, clf = fit_sample_data(trnsX, cont)

In [45]:
featImpMDI(fit, featNames=trnsX.columns)

,mean,std
I_0,0.036211,0.000441
I_1,0.025090,0.000202
I_2,0.029266,0.000265
I_3,0.028630,0.000282
I_4,0.037420,0.000506
I_5,0.035302,0.000473
I_6,0.029710,0.000284
I_7,0.034993,0.000430
I_8,0.041877,0.000507
I_9,0.029586,0.000257


## スニペット8.3 MDAによる特徴量重要度測定

In [46]:
# スニペット7.3, 7.4を利用
# from sklearn.model_selection import KFold
# class PurgedKFold(KFold):
#   '''
#   区間にまたがるラベルに対して機能するようにkFoldクラスを拡張する
#   訓練データのうちテストラベル区間と重複する観測値がパージされる
#   テストデータセットは連続的(shuffle=False)で、間に訓練データがないとする
#   '''

#   def __init__(self, n_splits=3, t1=None, pctEmbargo=0.):
#     if not isinstance(t1, pd.Series):
#       raise ValueError('Label Through Dates must be a pd.Series')
#     super(PurgedKFold, self).__init__(n_splits,shuffle=False,random_state=None)
#     self.t1=t1
#     self.pctEmbargo=pctEmbargo

#   def split(self,X,y=None,groups=None):
#     if (X.index==self.t1.index).sum()!=len(self.t1):
#       raise ValueError('X and ThruDateValues must have the same index')
#     indices=np.arange(X.shape[0])
#     mbrg=int(X.shape[0]*self.pctEmbargo)
#     test_starts=[(i[0],i[-1]+1) for i in np.array_split(np.arange(X.shape[0]),self.n_splits)]
#     for i,j in test_starts:
#       t0=self.t1.index[i] #テストデータセットの始まり
#       test_indices=indices[i:j]
#       maxT1Idx=self.t1.index.searchsorted(self.t1[test_indices].max())
#       train_indices=self.t1.index.searchsorted(self.t1[self.t1<=t0].index)
#       train_indices=np.concatenate((train_indices, indices[maxT1Idx+mbrg:]))
#     yield train_indices, test_indices


# def cvScore(clf,X,y,sample_weight,scoring='neg_log_loss',t1=None,cv=None,cvGen=None,pctEmbargo=None):
#   if scoring not in ['neg_log_loss', 'accuracy']:
#     raise Exception('wrong scoring method.')
#   from sklearn.metrics import log_loss, accuracy_score
#   # from clfSequential import PurgedKFold

#   if cvGen is None:
#     #パージ
#     cvGen=PurgedKFold(n_splits=cv,t1=t1,pctEmbargo=pctEmbargo)
#   score=[]
#   for train,test in cvGen.split(X = X):
#     fit = clf.fit(X = X.iloc[train, : ], y = y.iloc[train], sample_weight = sample_weight.iloc[train].values)
#     if scoring == 'neg_log_loss':
#       prob = fit.predict_proba(X.iloc[test, : ])
#       score_ = -1 * log_loss(y.iloc[test], prob, sample_weight = sample_weight.iloc[test].values, labels = clf.classes_)
#     else:
#       pred = fit.predict(X.iloc[test, : ])
#       score_ = accuracy_score(y.iloc[test], pred, sample_weight = sample_weight.iloc[test].values)
#     score.append(score_)
#   return np.array(score)

In [47]:
def featImpMDA(clf, X,y,cv,sample_weight, t1, pctEmbargo, scoring='neg_log_loss'):
    #アウトオブサンプルでのスコア低下に基づく特徴量重要度

    #スコアリングメソッドが間違っていたら弾く
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise Exception('wrong scoring method.')



    from sklearn.metrics import log_loss, accuracy_score

    #パージ付きcv（7章）
    cvGen = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
    scr0, scr1 = pd.Series(), pd.DataFrame(columns=X.columns)

    for i, (train, test) in enumerate(cvGen.split(X=X)):
        X0, y0, w0 = X.iloc[train,:], y.iloc[train], sample_weight.iloc[train]
        X1,y1,w1 = X.iloc[test,:], y.iloc[test], sample_weight.iloc[test]
        fit = clf.fit(X=X0, y=y0, sample_weight=w0.values)

        #並べ替え前のスコアをはかる
        if scoring == 'neg_log_loss':
            prob = fit.predict_proba(X1)
            scr0.loc[i] = -log_loss(y1, prob, sample_weight=w1.values, labels=clf.classes_)

        else:
            #scoring==accurateの場合
            pred = fit.predict(X1)
            scr0.loc[i] = accuracy_score(y1, pred, sample_weight = w1.values)

        # 並べ替え後のスコアをはかる
        for j in X.columns:
            X1_ = X1.copy(deep=True)
            np.random.shuffle(X1_[j].values) #1列並べ替え

            if scoring == 'neg_log_loss':
                prob = fit.predict_proba(X1_)
                scr1.loc[i,j] = -log_loss(y1, prob, sample_weight=w1.values, labels=clf.classes_)

            else:
                pred = fit.predict(X1_)
                scr1.loc[i,j] = accuracy_score(y1, pred, sample_weight=w1.values)

    imp = (-scr1).add(scr0, axis=0)

    if scoring == 'neg_log_loss':
        imp = imp/-scr1

    else:
        imp = imp/(1.-scr1)

    imp = pd.concat({'mean':imp.mean(), 'std': imp.std()*imp.shape[0]**-.5}, axis=1)

    return imp, scr0.mean()

人工データでやってみる

In [48]:
#featImpMDA(clf, X=trnsX, y=cont['bin'],cv=10,sample_weight=cont['w'], t1=cont['t1'],pctEmbargo=0)

# 8.4 代替効果を除いた特徴量重要度

## 8.4.1 単一特徴量重要度

In [49]:
#スニペット8.4 SFIの実装

def auxFeatImpSFI(featNames, clf, trnsX, cont, scoring, cvGen):
    imp = pd.DataFrame(columns=['means','std'])

    for featName in featNames:
        df0 = cvScore(clf, X=trnsX[[featName]],y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cvGen=cvGen)

        imp.iloc[featName, 'mean'] = df0.mean()
        imp.loc[featName, 'std'] = df0.std()*df0.shape[0]**-.5

    return imp

In [50]:
#人工データでやってみる



## 8.4.2 特徴量の直交化

In [51]:
#スニペット8.5 直交特徴量の計算

def get_eVec(dot, varThres):
    #dot prod行列からeVec(固有ベクトル)を計算し、次元を圧縮する
    eVal, eVec = np.linalg.eigh(dot)
    idx = eval.argsort()[::-1] # eVal(固有値)を降順にソートするための引数
    eVal, eVec = eVal[idx], eVec[:,idx]

    #2) 正の固有値のみ
    eVal = pd.Series(eVal, index=['PC_'+str(i+1) for i in range(eVal.shape[0])])
    eVec = pd.DataFrame(eVec, index=dot.index, columns=eVal.index)
    eVec = eVec.iloc[:, eVal.index]

    #3) 主成分によって次元削減する
    cumVar = eVal.cumsum()/eVal.sum()
    dim = cumVar.values.searchsorted(varThres)
    eVal, eVec = eVal.iloc[:dim+1], eVec.iloc[:,:dim+1]

    return eVal, eVec

In [52]:
def orthoFeats(dfX, varThres=.95):
    #特徴量のデータフレームdfXに対し、直交化した特徴量dfPを計算する
    dfZ = dfX.sub(dfX.mean(),axis=1).div(dfX.std(), axis=1) #標準化
    dot = pd.DataFrame(np.dot(dfZ.T, dfZ), index=dfX.columns, columns=dfX.columns)
    eVal, eVec = get_eVec(dot, varThres)
    dfP = np.dot(dfZ, eVec)
    return dfP

In [53]:
# スニペット8.6 特徴量重要度と逆PCA順位の間のケンドール加重タウの計算

from scipy.stats import weightedtau

featImp = np.array([.55, .33, .07, 0.5]) #特徴量重要度
pcRank = np.array([1,2,4,3]) #PCAランク
weightedtau(featImp, pcRank**-1.)[0] 


0.7333333333333332

# 8.6節

In [54]:
#7章のスニペット7.4

# def cvScore(clf,X,y,sample_weight,scoring='neg_log_loss',t1=None,cv=None,cvGen=None,pctEmbargo=None):
#   if scoring not in ['neg_log_loss', 'accuracy']:
#     raise Exception('wrong scoring method.')
#   from sklearn.metrics import log_loss, accuracy_score
#   # from clfSequential import PurgedKFold

#   if cvGen is None:
#     #パージ
#     cvGen=PurgedKFold(n_splits=cv,t1=t1,pctEmbargo=pctEmbargo)
#   score=[]
#   for train,test in cvGen.split(X = X):
#         fit = clf.fit(X = X.iloc[train, : ], y = y.iloc[train], sample_weight = sample_weight.iloc[train].values)
#         if scoring == 'neg_log_loss':
#             prob = fit.predict_proba(X.iloc[test, : ])
#             score_ = -1 * log_loss(y.iloc[test], prob, sample_weight = sample_weight.iloc[test].values, labels = clf.classes_)
#         else:
#             pred = fit.predict(X.iloc[test, : ])
#             score_ = accuracy_score(y.iloc[test], pred, sample_weight = sample_weight.iloc[test].values)
#         score.append(score_)
#   return np.array(score)

## スニペット8.8

デフォルトの分類器としてバギング決定木を使用

３つの特徴量重要度手法が機能しているかどうか評価するため、同じデータセットに対して各分析を実行できる関数を定義する

In [55]:
def featImportance(trnsX, cont, n_estimators=1000, cv=10, max_samples=1., numThreads=24, pctEmbargo=0, scoring='accuracy', method='SFI', minWLeaf=0., **kargs):

    #20章のmpEngineを利用する
    #mpEngine.pyがワーキングディレクトリにあればインポートできる
    from mpEngine import mpPandasObj

    n_jobs = (-1 if numThreads>1 else 1) #dirac1のht_helperの場合は1スレッドのみをはしらせる

    # 1) classifierとcvを用意する。
    # マスキングを避けるためにmax_features=1とする

    clf = DecisionTreeClassifier(criterion='entropy', max_features=1, class_weight='balanced', min_weight_fraction_leaf=minWLeaf)

    #base_estimatorはestimatorに変わったので、バージョンに合わせる
    clf = BaggingClassifier(base_estimator=clf, n_estimators=n_estimators, max_features=1., max_samples=max_samples, oob_score=True, n_jobs=n_jobs)
    #clf = BaggingClassifier(estimator=clf, n_estimators=n_estimators, max_features=1., max_samples=max_samples, oob_score=True, n_jobs=n_jobs)

    fit = clf.fit(X = trnsX, y=cont['bin'], sample_weight=cont['w'].values)
    oob = fit.oob_score_

    if method == 'MDI':
        imp = featImpMDI(fit, featNames=trnsX.columns)

        #7章のスニペット7.4を使用している
        oos = cvScore(clf, X=trnsX, y=cont['bin'], cv=cv, sample_weight=cont['w'], t1=cont['t1'],pctEmbargo=pctEmbargo, scoring=scoring).mean()

    elif method == 'MDA':
        imp, oos = featImpMDA(clf, X=trnsX, y=cont['bin'], cv=cv, sample_weight=cont['w'], t1=cont['t1'], pctEmbargo=pctEmbargo, scoring=scoring)

    elif method == 'SFI':
        cvGen = PurgedKFold(n_splits=cv, t1=cont['t1'], pctEmbargo=pctEmbargo)
        oos = cvScore(clf, X=trnsX, y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cvGen=cvGen).mean()
        clf.n_jobs=1 # clfではなくauxFeatImpSFIを並列化する
        imp=mpPandasObj(auxFeatImpSFI, ('featNames', trnsX.columns), numThreads, clf=clf, trnsX=trnsX, cont=cont, scoring=scoring,cvGen=cvGen)

    return imp, oob, oos

In [56]:
#featImportance(trnsX, cont, n_estimators=10)

In [57]:
#スニペット8.10 特徴量重要度プロット関数
    #スニペット8.9で使用

def plotFeatImportance(pathOut, imp, oob, oos, method, tag=0, simNum=0, **kargs):
    #impの平均値バーを標準偏差によってプロットする
    mpl.figure(figsize=(10, imp.shape[0]/5.))
    imp = imp.sort_values('mean', ascending=True)
    ax=imp['mean'].plot(kind='barh', color='b', alpha=0.25, xerr=imp['std'], error_kw={'ecolor':'r'})

    if method == 'MDI':
        mpl.xlim([0,imp.sum(axis=1).max()])
        mpl.axvline(1./imp.shape[0], linewidth=1, color='r', linestyle='dotted')
    
    ax.get_yaxis().set_visible(False)

    for i,j in zip(ax.patches, imp.index):
        ax.text(i.get_width()/2, i.get_y() + i.get_height()/2, j, ha='center', va='center', color='black')
    
    #mpl.title('tag=' + tag + ' | simNum=' + str(simNum) + ' | oob=' + str(round(oob,4)) + ' | oos=' + str(round(oos, 4)))
    mpl.savefig(pathOut + 'featImportance_' + str(simNum) + '.png', dpi=100)
    mpl.clf()
    mpl.close()

    return

In [58]:
#スニペット8.9 全要素の呼び出し
def testFunc(n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10):
    #人工データにおける特徴量重要度関数のパフォーマンスを測定する
    #ノイズ比率、すなわちノイズ特徴量=n_features-n_informative-n_redundantである

    trnsX, cont = getTestData(n_features, n_informative, n_redundant, n_samples)
    dict0 = {'minWLeaf':[0.], 'scoring':['accuracy'], 'method':['MDI', 'MDA', 'SFI'], 'max_samples':[1.]}


    #jobs, out = (dict(izip(dict0, i)) for i in product(*dict0.values())),[]
    jobs, out = (dict(zip(dict0, i)) for i in itertools.product(*dict0.values())),[]
    kargs = {'pathOut':'./testFunc/', 'n_estimators':n_estimators, 'tag':'testFunc', 'cv':cv}

    for job in jobs:
        job['simNum'] = job['method'] + '_' + job['scoring'] + '_' + '%.2f'%job['minWLeaf'] + '_' + str(job['max_samples'])
        print(job['simNum'])

        kargs.update(job)
        imp, oob, oos = featImportance(trnsX=trnsX, cont=cont,**kargs)
        plotFeatImportance(imp=imp, oob=oob, oos=oos, **kargs)

        df0 = imp[['mean']]/imp['mean'].abs().sum()
        df0['type'] = [i[0] for i in df0.index]
        df0 = df0.groupby('type')['mean'].sum().to_dict()
        df0.update({'oob':oob, 'oos': oos})
        df0.update(job)
        out.append(df0)

    out = pd.DataFrame(out).sort_values(['method','scoring','minWLeaf', 'max_samples'])
    out=out['method', 'scoring', 'minWLeaf', 'max_samples', 'I', 'R', 'N', 'oob', 'oos']
    out.to_csv(kargs['pathOut'] + 'stats.csv')
    
    return
        


実行に時間がかかるため、n_estimators=100, n_samples=1000にしてデータや試行回数を減らしている。
それでもかなりの時間がかかった。

In [59]:
testFunc(n_estimators=100, n_samples=1000)

MDI_accuracy_0.00_1.0


MDA_accuracy_0.00_1.0


c:\Users\sinya\.conda\envs\blockchainpy37\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
c:\Users\sinya\.conda\envs\blockchainpy37\lib\site-packages\matplotlib\axes\_axes.py:1066: RuntimeWarning: All-NaN axis encountered
  minx = np.nanmin(masked_verts[..., 0])
c:\Users\sinya\.conda\envs\blockchainpy37\lib\site-packages\matplotlib\axes\_axes.py:1067: RuntimeWarning: All-NaN axis encountered
  maxx = np.nanmax(masked_verts[..., 0])


SFI_accuracy_0.00_1.0
